# Instructions

1. Create a tarball that contains all the structures you want to put in the database:
> tar -czf random_name.tar.gz struct1.cif struct2.cif
<br>
or
<br>
> tar -czf random_name.tar.gz \*.cif

2. Hit "Select tarball" button and chose the random_name.tar.gz. Then press Open
3. After the previous step the button "Store structures" will be activated. Press it
4. The list of apploaded CIF files will be shown
5. If you need more files, start again from the step 1 or 2

###  <font color='red'> Warning1: for the moment the application works with CIF files only. </font> 
###  <font color='red'> Warning2: do not put structures into a subfolder. Do exactly as it is described in the step 1. </font> 

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.cif import CifData
from aiida.orm.calculation import Calculation

import ase.io
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget
from tempfile import NamedTemporaryFile
import tarfile

In [ ]:
tarball = None

## Upload file

In [ ]:
def on_file_upload(c):
    global tarball
    tarball = NamedTemporaryFile(suffix=file_upload.filename)
    f = open(tarball.name, "w")
    f.write(file_upload.data)
    f.close()
    upload_out.value = 'Success.\nNow hit "Store structures" button.'
    btn_store.disabled = False

#TODO: FileUploadWidget doesn't fire event when same file is uploaded twice
file_upload = FileUploadWidget("Select tarball")
file_upload.observe(on_file_upload, names='data')
upload_out = ipw.HTML('Please select a tarbal with CIF files')

display(file_upload, upload_out)

## Store in AiiDA Database

In [ ]:
def on_click_store(b):
    global tarball
    btn_store.disabled = True
    with store_out:
        clear_output()
        tar = tarfile.open(tarball.name)
        count = 0
        for member in tar.getmembers():
            count += 1
            fcif = tar.extractfile(member)
            cif = NamedTemporaryFile(prefix='', suffix=member.name)
            print (cif.name)
            f = open(cif.name, "w")
            f.write(fcif.read())
            f.close()
            atoms_cif = CifData(file=cif.name, scan_type='flex', parse_policy='lazy')
            atoms_cif.store()
        html_out.value = "{} structures stored".format(count)

btn_store = ipw.Button(description='Store structures')
btn_store.on_click(on_click_store)
btn_store.disabled = True
html_out = ipw.HTML("")


store_out = ipw.Output()
display(btn_store, html_out, store_out)